# Setup

In [1]:
from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)

project root: /Users/joachimpfefferkorn/repos/neurovolume


# Test registration with one frame
Somewhat adapted from a GPT response, let's first make sure that our registration applies to one frame.

In [2]:
bold_image = ants.image_read(bold_stim_filepath)  # Replace with your BOLD image path
t1_image = ants.image_read(t1_input_filepath)     # Replace with your T1 image path

In [3]:
test_slice = ants.from_numpy(bold_image.numpy()[:,:,:,0], spacing=bold_image.spacing[:3])

In [4]:
print("Test Slice Direction:\n", test_slice.direction)
print("T1 Image Direction:\n", t1_image.direction)
print("Test Slice Origin:", test_slice.origin)
print("T1 Image Origin:", t1_image.origin)

Test Slice Direction:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
T1 Image Direction:
 [[-0.99999677  0.00254     0.        ]
 [-0.00254    -0.99999677  0.        ]
 [ 0.          0.          1.        ]]
Test Slice Origin: (0.0, 0.0, 0.0)
T1 Image Origin: (119.98899841308594, 104.5199966430664, -84.24569702148438)


Oooh, these are different directions and origins! Thanks GPT!

In [5]:
# test_slice.set_direction(t1_image.direction)
# test_slice.set_origin(t1_image.origin)

In [6]:
# slice_registration = ants.registration(
#     fixed=t1_image,
#     moving=test_slice,
#     type_of_transform='Affine'  # You can also use 'Affine' or 'SyN' for deformable registration
# )

# bold_slice_registered = ants.apply_transforms(
#     fixed=t1_image,
#     moving=test_slice,
#     transformlist=slice_registration['fwdtransforms'],
#     interpolator='linear'
# )

In [7]:
def compare_bold_alignment(bold_frame_vol: np.ndarray, anat_vol: np.ndarray, cmap=default_cmap):
    #Just doing the Z dimension for now
    #No time, just a BOLD frame
    def x_coord(slice_idx, opacity):
        fig, axes = plt.subplots(1,3, figsize=(15,5))

        axes[0].imshow(bold_frame_vol[:,:,slice_idx], cmap='hot')
        axes[0].set_title('BOLD')

        axes[1].imshow(anat_vol[:,:,slice_idx], cmap='gray')
        axes[1].set_title('Anatomy')

        axes[2].imshow(anat_vol[:,:,slice_idx], cmap='gray')
        axes[2].imshow(bold_frame_vol[:,:,slice_idx], cmap='hot', alpha=opacity)
        axes[2].set_title('Overlay')
    interact(x_coord, slice_idx=(0, anat_vol.shape[2]-1), opacity=(0, 1.0))

In [8]:
# compare_bold_alignment(bold_slice_registered.numpy(), t1_image.numpy())

This looks pretty great!

# Previous Failures
This works! but many of the following things we have tried, have not:

## Canonical Way, register all frames within one function

The following does not (it crashes the kernel):

````python
bold_registered = ants.apply_transforms(
    fixed=t1_image,
    moving=bold_image,
    transformlist=registration['fwdtransforms'],
    interpolator='linear',
    imagetype=3
)
````

I suspect this has to do with a glitch on the image type 3 as this was also an issue yesterday. Perhaps open an issue on github

As a workaround, let's use a loop to cycle through all the 4th dimensional stuff.

Given the computational lift of registration, let's write them to a list

## Gathering the Transforms, then registering
The following code proved problematic:
````python
registrations = []
for frame in range(bold_image.shape[3]):
    print(f"frame {frame}/{bold_image.shape[3]}")
    print(" creating bold frame")
    bold_frame = ants.from_numpy(bold_image.numpy()[:,:,:,frame])

    print(" Registering bold frame to T1 image")
    registration = ants.registration(
    fixed=t1_image,
    moving=bold_frame,
    type_of_transform='Rigid'  # You can also use 'Affine' or 'SyN' for deformable registration
    )
    registrations += registration #This is incorrect! It's just adding the var names
````
When trying to apply these registrations, I got strange behavior. I believe this is because these `registration` values might be getting garbled behind the scenes somewhere. To get around this, let's see if one big function that gathers registrations and applies them does the trick!

Also, perhaps you could have made this a dictionary `img:transforms`

# 4D Registration

In order to test this, let's build a very short slice of the BOLD volume

In [ ]:
def align_bold_to_t1(bold_img, t1_img):
    result_np = np.empty((t1_image.shape[0], t1_image.shape[1], t1_image.shape[2], bold_img.shape[3]), dtype=np.float32) #you did change this from bold_image to bold_img right after it mysteriously worked, just fyi
    template_bold = bold_img
    for frame in range(bold_img.shape[3]):
            print(f"frame {frame}/{bold_img.shape[3]}")
            print(" creating bold frame")
            bold_frame = ants.from_numpy(bold_img.numpy()[:,:,:,frame], spacing=bold_img.spacing[:3])
            print(" Setting direction and origin")
            bold_frame.set_direction(t1_img.direction)
            bold_frame.set_origin(t1_img.origin)
            print(" Registering bold frame to T1 image")
            frame_registration = ants.registration(
                fixed=t1_img,
                moving=bold_frame,
                type_of_transform='Affine'  # You can also use 'Affine' or 'SyN' for deformable registration
            )
            print(" Applying transforms")
            bold_slice_registered = ants.apply_transforms(
                fixed=t1_img,
                moving=bold_frame,
                transformlist=frame_registration['fwdtransforms'],
                interpolator='linear'
            )
            print(" Adding BOLD frame to np volume")
            #DEBUG STUFF
            print(f"    bold_slice_registered.numpy() shape {bold_slice_registered.numpy().shape}")
            print(f"    result_np[:,:,:,frame] shape {result_np[:,:,:,frame].shape}")
            print(f"    bold_slice_registered.numpy().dtype: {bold_slice_registered.numpy().dtype}")
            print(f"    result_np.dtype: {result_np.dtype}")
            result_np[:,:,:,frame] = bold_slice_registered.numpy()
            print(f"Updating template_bold")
            template_bold = bold_slice_registered
    print("All BOLD frames registered and stored in numpy arrays. Creating final image")
    bold_registered_img = ants.from_numpy(result_np, origin=template_bold.origin, spacing=template_bold.spacing, direction=template_bold.direction)
    print("done")
    return bold_registered_img

In [10]:
sliced = bold_image.numpy()[:, :, :, :2]
bold_truncated_img = ants.from_numpy(sliced, spacing=bold_image.spacing, origin=bold_image.origin, direction=bold_image.direction)
#hopefully the above args match all the metadata. I've printed to check but there might be some stuff hidden away
print(bold_truncated_img)
print(bold_image)

ANTsImage
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (64, 64, 35, 2)
	 Spacing    : (4.0, 4.0, 4.0, 2.0)
	 Origin     : (-127.953, 108.933, -74.8393, 0.0)
	 Direction  : [ 1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]

ANTsImage
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (64, 64, 35, 185)
	 Spacing    : (4.0, 4.0, 4.0, 2.0)
	 Origin     : (-127.953, 108.933, -74.8393, 0.0)
	 Direction  : [ 1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]



In [11]:
aligned_bold_seq = align_bold_to_t1(bold_truncated_img, t1_image) #erroneously labeled np, these are in fact images

frame 0/2
 creating bold frame
 Setting direction and origin
 Registering bold frame to T1 image
 Applying transforms
 Adding BOLD frame to np volume
    bold_slice_registered.numpy() shape (512, 512, 296)
    result_np[:,:,:,frame] shape (512, 512, 296)
    bold_slice_registered.numpy().dtype: float32
    result_np.dtype: float32
frame 1/2
 creating bold frame
 Setting direction and origin
 Registering bold frame to T1 image
 Applying transforms
 Adding BOLD frame to np volume
    bold_slice_registered.numpy() shape (512, 512, 296)
    result_np[:,:,:,frame] shape (512, 512, 296)
    bold_slice_registered.numpy().dtype: float32
    result_np.dtype: float32
All BOLD frames registered and stored in numpy arrays. Creating final image


NameError: name 'template_bold' is not defined